# 5. The Vector API: tipg

<https://developmentseed.org/tipg>

No geospatial data stack is complete without a mechanism for serving vector features as GeoJSON or for consumption in web maps as vector tiles. **tipg** is the component that fills that niche in eoAPI.

From the tipg README:
> `tipg`, pronounced *T[ee]pg*, is a **Python** package that helps create lightweight OGC **Features** and **Tiles** API with a PostGIS Database backend. The API has been designed for [OGC Features](https://ogcapi.ogc.org/features) and [OGC Tiles](https://ogcapi.ogc.org/tiles/) specifications.

In addition to serving existing features from a pre-defined set of tables in a PostGIS-enabled PostgreSQL database, it can serve features from custom views defined in user-defined PostgreSQL functions.

## 5.1 Configuration

In an eoAPI stack, tipg can be connected to any PostgreSQL database including the existing pgstac database. This is controlled by the `POSTGRES_*` environment variables in the application runtime.

When deploying tipg, you can specify which schemas in your database will be exposed to the tipg API. This is controlled by the `TIPG_DB_SCHEMAS` environment variable.

For the workshop we have created a schema in the pgstac database called `features` that you will be working with. To expose this schema to tipg we set `TIPG_DB_SCHEMAS=["features"]` in the application runtime (see line 178 in [infrastructure/app.py](../infrastructure/app.py)). There is one table that has been pre-loaded (`features.terrestrial_ecoregions`) for the examples in this notebook.

### Additional resources:
- tipg configuration docs: <https://developmentseed.org/tipg/user_guide/configuration>

## 5.2 OGC API - Features
tipg contains an OGC Features API that is interoperable with many existing applications.

<https://ogcapi.ogc.org/features/overview.html>

Each table in the PostgreSQL schema represents a single collection. The list of collections available to tipg can be obtained with a GET request to the `/collections` endpoint

In [1]:
import json
import os

import httpx

tipg_endpoint = os.getenv("TIPG_API_ENDPOINT")

collections_request = httpx.get(f"{tipg_endpoint}/collections")

print(json.dumps(collections_request.json(), indent=2))

{
  "links": [
    {
      "href": "https://ea1xibo0hd.execute-api.us-west-2.amazonaws.com/collections",
      "rel": "self",
      "type": "application/json"
    }
  ],
  "numberMatched": 1,
  "numberReturned": 1,
  "collections": [
    {
      "id": "features.terrestrial_ecoregions",
      "title": "features.terrestrial_ecoregions",
      "links": [
        {
          "href": "https://ea1xibo0hd.execute-api.us-west-2.amazonaws.com/collections/features.terrestrial_ecoregions",
          "rel": "collection",
          "type": "application/json"
        },
        {
          "href": "https://ea1xibo0hd.execute-api.us-west-2.amazonaws.com/collections/features.terrestrial_ecoregions/items",
          "rel": "items",
          "type": "application/geo+json"
        },
        {
          "href": "https://ea1xibo0hd.execute-api.us-west-2.amazonaws.com/collections/features.terrestrial_ecoregions/queryables",
          "rel": "queryables",
          "type": "application/schema+json"
       

Each collection has a set of links associated with it including:
- `/collections/{collection_id}/queryables`: describes the fields that can be used for filtering features
- `/collections/{collection_id}/items`: where features can be accessed
- `/collections/{collection_id}/tiles`: list of tile matrix set IDs that are available for tile requests
- `/collections/{collection_id}/tiles/{tileMatrixSetId}`: returns a tilejson for a vector tile layer
- `/collections/{collection_id}/tiles/{tileMatrixSetId}/viewer`: interactive map of the collection

The `/items`, `/tiles/{tileMatrixSetId}`, and `/tiles/{tileMatrixSetId}/viewer` endpoints will all accept field filters in the form of `{queryable}={value}` where `queryable` is one of the fields listed in the `/queryables` response for that collection.

### 5.2.2 /collections/{collection_id/queryables

The `/queryables` endpoint returns a list of fields that can be used to filter features in requests for a collection:

In [2]:
collection_id = "features.terrestrial_ecoregions"

queryables_request = httpx.get(
    f"{tipg_endpoint}/collections/{collection_id}/queryables"
)

print(json.dumps(queryables_request.json(), indent=2))

{
  "title": "features.terrestrial_ecoregions",
  "properties": {
    "geom": {
      "$ref": "https://geojson.org/schema/MultiPolygon.json"
    },
    "area": {
      "name": "area",
      "type": "number"
    },
    "area_km2": {
      "name": "area_km2",
      "type": "number"
    },
    "biome": {
      "name": "biome",
      "type": "number"
    },
    "eco_code": {
      "name": "eco_code",
      "type": "string"
    },
    "eco_id": {
      "name": "eco_id",
      "type": "number"
    },
    "eco_name": {
      "name": "eco_name",
      "type": "string"
    },
    "eco_num": {
      "name": "eco_num",
      "type": "number"
    },
    "eco_sym": {
      "name": "eco_sym",
      "type": "number"
    },
    "g200_biome": {
      "name": "g200_biome",
      "type": "number"
    },
    "g200_num": {
      "name": "g200_num",
      "type": "number"
    },
    "g200_regio": {
      "name": "g200_regio",
      "type": "string"
    },
    "g200_stat": {
      "name": "g200_stat",
      

### 5.2.2 /collections/{collection_id}/items

The `/items` endpoint for a collection can be used to retrieve paginated lists of features in a number of formats:
  - GeoJSON
  - CSV
  - JSON
  - GeoJSON Sequence
  - NDJSON (new-line-delimited JSON)
  - HTML (for viewing in a browser)

[source](https://github.com/developmentseed/tipg/blob/1a2e5eb6816d51f97ae1d5bbc1d1e952d996987b/tipg/factory.py#L757-L762)

Try retrieving a page of results from the `features.terrestrial_ecoregions` collection. This will return a GeoJSON FeatureCollection with two features (`limit=2`):

In [3]:
geojson_request = httpx.get(
    f"{tipg_endpoint}/collections/{collection_id}/items",
    params={"f": "geojson", "limit": 2},
)
geojson_response = geojson_request.json()
print(json.dumps(geojson_response, indent=2))

{
  "type": "FeatureCollection",
  "id": "features.terrestrial_ecoregions",
  "title": "features.terrestrial_ecoregions",
  "description": "features.terrestrial_ecoregions",
  "numberMatched": 14458,
  "numberReturned": 2,
  "links": [
    {
      "title": "Collection",
      "href": "https://ea1xibo0hd.execute-api.us-west-2.amazonaws.com/collections/features.terrestrial_ecoregions",
      "rel": "collection",
      "type": "application/json"
    },
    {
      "title": "Items",
      "href": "https://ea1xibo0hd.execute-api.us-west-2.amazonaws.com/collections/features.terrestrial_ecoregions/items?f=geojson&limit=2",
      "rel": "self",
      "type": "application/geo+json"
    },
    {
      "href": "https://ea1xibo0hd.execute-api.us-west-2.amazonaws.com/collections/features.terrestrial_ecoregions/items?f=geojson&limit=2&offset=2",
      "rel": "next",
      "type": "application/geo+json",
      "title": "Next page"
    }
  ],
  "features": [
    {
      "type": "Feature",
      "geome

You can request a sequence of GeoJSON features separated by new lines with `f=geojsonseq`

In [4]:
geojsonseq_request = httpx.get(
    f"{tipg_endpoint}/collections/{collection_id}/items",
    params={"f": "geojsonseq", "limit": 2},
)

print(geojsonseq_request.text)

{"type":"Feature","geometry":{"type":"MultiPolygon","coordinates":[[[[-112.269721533,29.326477674],[-112.288085612,29.326271814],[-112.30207065,29.321869806],[-112.313644549,29.320189905],[-112.320777549,29.316841167],[-112.324989623,29.315979172],[-112.329490537,29.317375095],[-112.33276367,29.320391909],[-112.336730658,29.320041043],[-112.343643548,29.314392143],[-112.347297555,29.315589079],[-112.349532674,29.318044138],[-112.350349574,29.320272048],[-112.350341527,29.322913856],[-112.348998579,29.328520511],[-112.346343527,29.333462314],[-112.343353535,29.34005401],[-112.337051685,29.34927578],[-112.334724533,29.353559603],[-112.332076522,29.357179579],[-112.32678268,29.362443415],[-112.323142587,29.365402226],[-112.318191564,29.367363927],[-112.310600577,29.367335931],[-112.305328528,29.366327086],[-112.30072066,29.364000269],[-112.288543599,29.356033438],[-112.281631546,29.352044658],[-112.276695611,29.347406951],[-112.274070567,29.344095931],[-112.270789555,29.338803095],[-112.2

You can apply a filter using the fields returned in the `/queryables` endpoint for a collection:

In [5]:
filtered_request = httpx.get(
    f"{tipg_endpoint}/collections/{collection_id}/items",
    params={
        "eco_name": "Northern Mesoamerican Pacific mangroves",
        "f": "geojson",
        "limit": 2,
    },
)

filtered_response = filtered_request.json()
print(
    f"{filtered_response['numberMatched']} features match this filtered request",
    f"out of {geojson_response['numberMatched']} features in the entire collection",
)

77 features match this filtered request out of 14458 features in the entire collection


In addition to field-based filters, you can use other standard filter mechanisms:
- `ids`: limit to a comma-separated list of feature ids
- `bbox`: filter by bounding box
- `datetime`: filter by datetime (use with `datetime-column` parameter)
- `filter`: apply a CQL2 filter (use with `filter-lang` parameter set to cql2-text or cql2-json) for more complex filter operations

In [6]:
# filter by bounding box
bbox_filtered_request = httpx.get(
    f"{tipg_endpoint}/collections/{collection_id}/items",
    params={"bbox": "-77,39,-76,40", "f": "geojson", "limit": 2},
)

bbox_filtered_response = bbox_filtered_request.json()
print(
    f"{bbox_filtered_response['numberMatched']} features match this filtered request",
    f"out of {geojson_response['numberMatched']} features in the entire collection",
)

5 features match this filtered request out of 14458 features in the entire collection


tipg also comes with a convenient HTML response type which makes it possible to interact with the endpoints in your browser. The returned geojson features from a `/items` request will be displayed in a map!

In [7]:
from IPython.display import IFrame

bbox_filtered_request = httpx.get(
    f"{tipg_endpoint}/collections/{collection_id}/items",
    params={
        "bbox": "-77,39,-76,40",
        "f": "html",
    },
)

IFrame(
    bbox_filtered_request.url,
    width=1200,
    height=800,
)

Here is a view of the full API docs for the `/collections/{collection_id}/items` endpoint:

In [8]:
IFrame(
    f"{tipg_endpoint}/api.html#OGC Features API/items_collections__collectionId__items_get",
    width=1200,
    height=800,
)

## 5.3 OGC API - Tiles

tipg also serves an OGC Tiles API for vector tiles.

<https://ogcapi.ogc.org/tiles/>

The Tiles API works exactly like the Features API but instead of taking requests for entire features it accepts requests for XYZ vector tiles that are very useful for streaming vector data into map client applications. This is useful because it will becomes impracticalimpossible to stream all of a collection's features into a map application as a geojson - tipg moves the simplification and filtering operations up to the PostGIS database and returns the minimum required data to the map client.

### 5.3.1 /collections/{collection_id/tiles/{tileMatrixSetId}/tilejson.json

The `tilejson` endpoint is the most useful for adding vector tile layers to a map application. The response contains information about the available fields (which can be used for styling the vector tiles), the full collection extent, and the XYZ tile url that can be loaded as a layer in a map.

In [9]:
tilejson_request = httpx.get(
    f"{tipg_endpoint}/collections/{collection_id}/tiles/WebMercatorQuad/tilejson.json",
)
tilejson_response = tilejson_request.json()
print(json.dumps(tilejson_response, indent=2))

{
  "tilejson": "3.0.0",
  "name": "features.terrestrial_ecoregions",
  "version": "1.0.0",
  "scheme": "xyz",
  "tiles": [
    "https://ea1xibo0hd.execute-api.us-west-2.amazonaws.com/collections/features.terrestrial_ecoregions/tiles/WebMercatorQuad/{z}/{x}/{y}"
  ],
  "vector_layers": [
    {
      "id": "default",
      "fields": {
        "area": "number",
        "area_km2": "number",
        "biome": "number",
        "eco_code": "string",
        "eco_id": "number",
        "eco_name": "string",
        "eco_num": "number",
        "eco_sym": "number",
        "g200_biome": "number",
        "g200_num": "number",
        "g200_regio": "string",
        "g200_stat": "number",
        "gbl_stat": "number",
        "id": "number",
        "objectid": "number",
        "per_area": "number",
        "per_area_1": "number",
        "per_area_2": "number",
        "perimeter": "number",
        "realm": "string",
        "shape_area": "number",
        "shape_leng": "number"
      },
  

All of the same rules for queryables and query parameters from the `/items` endpoint apply to the `/tiles` endpoints, too. The query parameters will be tacked onto the end of the XYZ tile URL:

In [10]:
filtered_tilejson_request = httpx.get(
    f"{tipg_endpoint}/collections/{collection_id}/tiles/WebMercatorQuad/tilejson.json",
    params={
        "eco_name": "Northern Mesoamerican Pacific mangroves",
    },
)
filtered_tilejson_response = filtered_tilejson_request.json()
print(json.dumps(filtered_tilejson_response, indent=2))

{
  "tilejson": "3.0.0",
  "name": "features.terrestrial_ecoregions",
  "version": "1.0.0",
  "scheme": "xyz",
  "tiles": [
    "https://ea1xibo0hd.execute-api.us-west-2.amazonaws.com/collections/features.terrestrial_ecoregions/tiles/WebMercatorQuad/{z}/{x}/{y}?eco_name=Northern+Mesoamerican+Pacific+mangroves"
  ],
  "vector_layers": [
    {
      "id": "default",
      "fields": {
        "area": "number",
        "area_km2": "number",
        "biome": "number",
        "eco_code": "string",
        "eco_id": "number",
        "eco_name": "string",
        "eco_num": "number",
        "eco_sym": "number",
        "g200_biome": "number",
        "g200_num": "number",
        "g200_regio": "string",
        "g200_stat": "number",
        "gbl_stat": "number",
        "id": "number",
        "objectid": "number",
        "per_area": "number",
        "per_area_1": "number",
        "per_area_2": "number",
        "perimeter": "number",
        "realm": "string",
        "shape_area": "nu

### 5.3.1 /collections/{collection_id/tiles/{tileMatrixSetId}/viewer

For a quick demonstration of how vector tiles enable visualization of massive feature collections, check out this map of the `terrestrial_ecoregions` table that lives in our database. It has 14,000+ features which we would never dream of downloading to view in a web map. Instead, we let our map client make requests for simplified features for each XYZ tile as we explore the map.

In [11]:
viewer_request = httpx.get(
    f"{tipg_endpoint}/collections/{collection_id}/tiles/WebMercatorQuad/viewer",
)

IFrame(
    viewer_request.url,
    width=1200,
    height=800,
)

You can apply a field-based filter to limit the features to a subset of the full collection:

In [12]:
filtered_viewer_request = httpx.get(
    f"{tipg_endpoint}/collections/{collection_id}/tiles/WebMercatorQuad/viewer",
    params={
        "eco_name": "Northern Mesoamerican Pacific mangroves",
    },
)

IFrame(
    filtered_viewer_request.url,
    width=1200,
    height=800,
)